In [ ]:
import time
import os

import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import *
tf.__version__

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from tensorflow.keras.metrics import Metric
class RSquare(Metric):
    """Compute R^2 score.
     This is also called as coefficient of determination.
     It tells how close are data to the fitted regression line.
     - Highest score can be 1.0 and it indicates that the predictors
       perfectly accounts for variation in the target.
     - Score 0.0 indicates that the predictors do not
       account for variation in the target.
     - It can also be negative if the model is worse.
     Usage:
     ```python
     actuals = tf.constant([1, 4, 3], dtype=tf.float32)
     preds = tf.constant([2, 4, 4], dtype=tf.float32)
     result = tf.keras.metrics.RSquare()
     result.update_state(actuals, preds)
     print('R^2 score is: ', r1.result().numpy()) # 0.57142866
    ```
    """

    def __init__(self, name='r_square', dtype=tf.float32):
        super(RSquare, self).__init__(name=name, dtype=dtype)
        self.squared_sum = self.add_weight("squared_sum", initializer="zeros")
        self.sum = self.add_weight("sum", initializer="zeros")
        self.res = self.add_weight("residual", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        self.squared_sum.assign_add(tf.reduce_sum(y_true**2))
        self.sum.assign_add(tf.reduce_sum(y_true))
        self.res.assign_add(
            tf.reduce_sum(tf.square(tf.subtract(y_true, y_pred))))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        mean = self.sum / self.count
        total = self.squared_sum - 2 * self.sum * mean + self.count * mean**2
        return 1 - (self.res / total)

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.squared_sum.assign(0.0)
        self.sum.assign(0.0)
        self.res.assign(0.0)
        self.count.assign(0.0)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = ((8/2.54), (6/2.54))
plt.rcParams["font.family"] = "Arial"
plt.rcParams["mathtext.default"] = "rm"
plt.rcParams.update({'font.size': 11})
MARKER_SIZE = 15
cmap_m = ["#f4a6ad", "#f6957e", "#fccfa2", "#8de7be", "#86d6f2", "#24a9e4", "#b586e0", "#d7f293"]
cmap = ["#e94d5b", "#ef4d28", "#f9a54f", "#25b575", "#1bb1e7", "#1477a2", "#a662e5", "#c2f442"]

plt.rcParams['axes.spines.top'] = False
# plt.rcParams['axes.edgecolor'] = 
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

# Model training

## hyperparameters

In [ ]:
SIZE = 100
BEST_PATH = './checkpoints/RNN_%dx%d' % (SIZE, SIZE)
DISP_STEPS = 100
TRAINING_EPOCHS = 500
BATCH_SIZE = 32
LEARNING_RATE = 0.001

## data loading

In [ ]:
l = np.load('./data/tot_dataset_%dx%d.npz' % (SIZE, SIZE))
raw_input = l['raw_input']
raw_label = l['raw_label']
test_input = l['test_input']
test_label = l['test_label']
MAXS = l['MAXS']
MINS = l['MINS']
MAXS = MAXS[:5]
MINS = MINS[:5]

SCREEN_SIZE = l['SCREEN_SIZE']

In [ ]:
print(raw_input.shape)
print(raw_label.shape)

In [ ]:
print(test_input.shape)
print(test_label.shape)

In [ ]:
raw_input = np.concatenate([raw_input[..., :5, 0], raw_input[..., :5, 1], raw_input[..., :5, 2], raw_input[..., :5, 3]], axis=-1)
raw_label = raw_label[..., :5, 0]
test_input = np.concatenate([test_input[..., :5, 0], test_input[..., :5, 1], test_input[..., :5, 2], test_input[..., :5, 3]], axis=-1)
test_label = test_label[..., :5, 0]

In [ ]:
print(raw_input.shape)
print(raw_label.shape)

In [ ]:
print(test_input.shape)
print(test_label.shape)

In [ ]:
raw_input = raw_input.astype(np.float32)
raw_label = raw_label.astype(np.float32)
test_input = test_input.astype(np.float32)
test_label = test_label.astype(np.float32)

In [ ]:
num_train = int(raw_input.shape[0]*.7)
raw_input, raw_label = shuffle(raw_input, raw_label, random_state=4574)
train_input, train_label = raw_input[:num_train, ...], raw_label[:num_train, ...]
val_input, val_label = raw_input[num_train:, ...], raw_label[num_train:, ...]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
train_dataset = train_dataset.cache().shuffle(BATCH_SIZE*50).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_input, val_label))
val_dataset = val_dataset.cache().shuffle(BATCH_SIZE*50).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_input, test_label))
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
class BiLSTMBlock(layers.Layer):
    def __init__(self, num_hidden):
        super(BiLSTMBlock, self).__init__()
        self.num_hidden = num_hidden
        
        self.bilstm = layers.Bidirectional(layers.LSTM(self.num_hidden, return_sequences=True))
        self.layernorm = layers.LayerNormalization()
        
    def call(self, inp):
        
        inp = self.bilstm(inp)
        inp = self.layernorm(inp)
        
        return inp

In [ ]:
class BiLSTM(Model):
    def __init__(self, out_len):
        super(BiLSTM, self).__init__()
        
        self.out_len = out_len
        
        self.bilstm_block1 = BiLSTMBlock(64)
        self.bilstm_block2 = BiLSTMBlock(64)
        self.ffnn = layers.Dense(self.out_len)
        
    def call(self, inp):
        output = self.bilstm_block1(inp)
        output = self.bilstm_block2(inp)
        output = self.ffnn(inp)
        
        return output

In [ ]:
#loss inputs should be masked.
loss_object = tf.keras.losses.MeanSquaredError()
def loss_function(model, inp, tar): #RNN specialized
    
    masked_real = tar * (1 - inp[..., 5:10])
    masked_pred = model(inp) * (1 - inp[..., 5:10])
    
    return loss_object(masked_real, masked_pred)

In [ ]:
bilstm_model = BiLSTM(test_label.shape[-1])
opt = tf.optimizers.Adam(learning_rate=LEARNING_RATE)

In [ ]:
@tf.function
def train(loss_function, model, opt, inp, tar):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss_function(model, inp, tar), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

In [ ]:
checkpoint_path = BEST_PATH

ckpt = tf.train.Checkpoint(bilstm_model=bilstm_model,
                           opt=opt)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=10)

In [ ]:
writer = tf.summary.create_file_writer('tmp')

# Training

In [ ]:
prev_test_loss = 100.0
with writer.as_default():
    with tf.summary.record_if(True):
        for epoch in range(TRAINING_EPOCHS):
            for step, (inp, tar) in enumerate(train_dataset):
                train(loss_function, bilstm_model, opt, inp, tar)
                loss_values = loss_function(bilstm_model, inp, tar)
                tf.summary.scalar('loss', loss_values, step=step)
                
                if step % DISP_STEPS == 0:
                    test_loss = 0
                    for step_, (inp_, tar_) in enumerate(test_dataset):
                        test_loss += loss_function(bilstm_model, inp_, tar_)
                        
                        if step_ > DISP_STEPS:
                            test_loss /= DISP_STEPS
                            break
                    if test_loss.numpy() < prev_test_loss:
                        ckpt_save_path = ckpt_manager.save()
                        prev_test_loss = test_loss.numpy()
                        print('Saving checkpoint at {}'.format(ckpt_save_path))
                    print('Epoch {} batch {} train loss: {:.4f} test loss: {:.4f}'
                          .format(epoch, step, loss_values.numpy(), test_loss.numpy()))

# Model evaluation

In [ ]:
i = -1
if ckpt_manager.checkpoints:
    ckpt.restore(ckpt_manager.checkpoints[i])
    print ('Checkpoint ' + ckpt_manager.checkpoints[i][-2:] +' restored!!')

In [ ]:
bilstm_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                   loss = tf.keras.losses.MeanSquaredError())

In [ ]:
test_loss = bilstm_model.evaluate(test_dataset)

In [ ]:
pred_result = bilstm_model.predict(test_dataset)

In [ ]:
print(pred_result.shape)
print(test_label.shape)

In [ ]:
masking = test_input[..., 5:10]

In [ ]:
masked_pred = np.ma.array(pred_result, mask=masking)
masked_label = np.ma.array(test_label, mask=masking)

In [ ]:
for _ in range(test_label.shape[-1]):
    print(np.sqrt(mean_squared_error(masked_label[..., _].reshape(-1), masked_pred[..., _].reshape(-1))))

In [ ]:
r2 = RSquare()
for _ in range(test_label.shape[-1]):
    r2.reset_states()
    print(r2(masked_label[..., _][~masked_label[..., _].mask].data.reshape(-1),
             masked_pred[..., _][~masked_pred[..., _].mask].data.reshape(-1)))

In [ ]:
plot_label = ((MAXS-MINS)*masked_label + MINS).reshape(-1, 1)
plot_label = plot_label[~plot_label.mask]
plot_pred = ((MAXS-MINS)*masked_pred + MINS).reshape(-1, 1)
plot_pred = plot_pred[~plot_pred.mask]
fig = plt.figure(figsize=((8/2.54)*4, (6/2.54)*4))
plt.scatter(plot_label, plot_pred, c=cmap[0], s=2, alpha=0.1)
plt.show()

r2 = RSquare()
r2.reset_states()
print(r2(plot_label, plot_pred))
print(np.sqrt(mean_squared_error(plot_label, plot_pred)))

In [ ]:
x_t = np.arange(0, test_label.shape[1])
for _ in range (3):

        NUMBERS = np.arange(1, pred_result.shape[0])
        np.random.shuffle(NUMBERS)
        NUMBERS = NUMBERS[:6]
        position = 331
        fig = plt.figure(figsize=((8.5/2.54)*8, (6/2.54)*8))
        
        i=0
        for NUMBER in NUMBERS:
            ax = plt.subplot(position)
            measured1 = plt.plot(x_t, test_label[NUMBER, :, i], c='k', alpha=0.8) #measured
            expect1 = plt.plot(x_t, masked_pred[NUMBER, :, i], 'o', c=cmap[5], alpha=0.4) #estimated
            expect1 = plt.plot(x_t, pred_result[NUMBER, :, i], c=cmap[2], alpha=0.4) #estimated
            ax.axis('off')

            position += 1
        plt.show()
        _ += 1

# Saving data

In [ ]:
plot_label = ((MAXS[:5]-MINS[:5])*masked_label[..., :5] + MINS[:5])
plot_label.fill_value = np.nan
plot_pred = ((MAXS[:5]-MINS[:5])*masked_pred[..., :5] + MINS[:5])
plot_pred.fill_value = np.nan

In [ ]:
f = open('./results/RNN_%d.npz' % SCREEN_SIZE, 'wb')
np.savez(f,
         test_label = plot_label.filled(),
         test_pred = plot_pred.filled()
        )
f.close()